# State-of-the-Art Automatic Speech Recognition (ASR) with Whisper

In previous classes, we built speech recognition from scratch - recording digits, training neural networks, and performing inference. That was incredible for understanding the fundamentals, but modern ASR systems can do *so much more*.

**Automatic Speech Recognition (ASR)** is the task of converting spoken language into text. Today's state-of-the-art systems can:

- Transcribe speech in **99+ languages**
- **Translate** speech from one language to English
- Provide **word-level timestamps**
- Handle accents, background noise, and multiple speakers
- Run on your **local machine** - no internet required!

### What you'll learn:

1. **How modern ASR works** - from audio waves to text (high-level)
2. **OpenAI's Whisper** - a powerful, free, open-source ASR model
3. **Transcription** - converting your speech to text with code
4. **Whisper's features** - language detection, translation, timestamps
5. **Real-time recognition** - live speech-to-text from your microphone
6. **Triggering actions** - using speech to control other things (OSC, Serial, etc.)
7. **Fine-tuning** (stretch goal) - customizing Whisper for specific tasks
8. **Other ASR systems** - what else is out there?

### Background Reading

- [Whisper Paper: Robust Speech Recognition via Large-Scale Weak Supervision](https://arxiv.org/abs/2212.04356) - OpenAI (2022)
- [Attention Is All You Need](https://arxiv.org/abs/1706.03762) - The Transformer paper (2017)
- [A Brief History of ASR](https://www.assemblyai.com/blog/the-history-of-speech-recognition-to-2022/) - AssemblyAI Blog

---

# Part 1: How Did We Get Here?

### A Brief History of Speech Recognition

| Year | System | What it could do |
|------|--------|------------------|
| 1952 | **AUDREY** (Bell Labs) | Recognized digits 0-9 from a single speaker |
| 1962 | **IBM Shoebox** | Recognized 16 words |
| 1970s | **HARPY** (CMU) | 1,011 word vocabulary using Hidden Markov Models |
| 1990s | **Dragon NaturallySpeaking** | First commercial dictation software |
| 2011 | **Siri** (Apple) | Voice assistant using deep learning |
| 2014 | **Deep Speech** (Baidu) | End-to-end deep learning for ASR |
| 2020 | **Wav2Vec 2.0** (Meta) | Self-supervised learning for speech |
| 2022 | **Whisper** (OpenAI) | Trained on 680,000 hours of multilingual audio |
| 2024+ | **Canary, Voxtral, etc.** | Even more accurate, multilingual, real-time |

In Class 2, we built our own "Audrey" - a digit recognizer trained on our own voice. Now we'll use **Whisper**, which was trained on 680,000 hours of audio from the internet and can understand almost any spoken language!

## How Modern ASR Works (High Level)

Modern ASR systems like Whisper use a **Transformer** architecture - the same type of neural network behind ChatGPT, but adapted for audio.

### The Pipeline

```
Your Voice -> Microphone -> Audio Waveform -> Mel Spectrogram -> Encoder -> Decoder -> Text
```

Let's break this down:

1. **Audio Waveform**: Sound captured as numbers (like we learned in Class 1!)
2. **Mel Spectrogram**: Converts the waveform into a visual representation of frequencies over time (also from Class 1!). This is the input to the neural network.
3. **Encoder**: A neural network that reads the spectrogram and creates a rich "understanding" of the audio. Think of it like someone listening carefully and forming a mental picture of the sounds.
4. **Decoder**: Another neural network that generates text one word (or "token") at a time, guided by the encoder's understanding. Like translating that mental picture into written words.

### What's Special About Whisper?

| Feature | Details |
|---------|---------|
| **Training Data** | 680,000 hours of audio from the internet |
| **Languages** | 99+ languages supported |
| **Multitask** | Transcription, translation, language detection, timestamps |
| **Open Source** | Free to download, run, and modify! |
| **Runs Locally** | No internet connection required after downloading the model |
| **Model Sizes** | From tiny (39M params) to large (1.5B params) - pick what fits your computer |

### How Whisper Was Trained

Unlike our Audrey model (trained on 10 recordings!), Whisper was trained on an enormous amount of data scraped from the internet. The key insight: OpenAI collected audio that *already had* subtitles or transcriptions (like YouTube videos with captions), so they didn't need humans to manually label everything. This is called **weak supervision** - the labels aren't perfect, but there are *so many* of them that the model still learns incredibly well.

# Part 2: Setup and Installation

Before we can use Whisper, we need to install it and a system dependency called **ffmpeg**.

### Step 1: Install ffmpeg (audio processing tool)

Whisper uses ffmpeg under the hood to read and process audio files. Install it for your operating system:

**Mac** (using Homebrew):
```bash
brew install ffmpeg
```

**Windows** (using Chocolatey):
```bash
choco install ffmpeg
```

**Windows** (manual): Download from [ffmpeg.org](https://ffmpeg.org/download.html) and add to your system PATH.

**Linux**:
```bash
sudo apt install ffmpeg
```

### Step 2: Install the openai-whisper Python package

```bash
uv pip install openai-whisper
```

That's it! Let's verify everything is set up.

In [ ]:
# Check if ffmpeg is installed (required by Whisper for audio processing)
# If this shows a warning, follow the installation instructions above!
#
# NOTE: Jupyter notebooks sometimes can't find programs that your terminal can.
# This is because the notebook's PATH (where it looks for programs) is different
# from your terminal's PATH. The code below checks common locations and fixes this.

import shutil
import subprocess
import os
import sys
import platform

# Common locations where ffmpeg might be installed
# Jupyter notebooks often don't inherit your full shell PATH, so we check manually
common_paths = [
    "/opt/homebrew/bin",      # macOS Apple Silicon (M1/M2/M3) Homebrew
    "/usr/local/bin",         # macOS Intel Homebrew / Linux
    "/usr/bin",               # Linux system install
    os.path.expanduser("~/bin"),  # User local bin
]

# Add common paths to PATH if they're not already there
for p in common_paths:
    if os.path.isdir(p) and p not in os.environ.get("PATH", ""):
        os.environ["PATH"] = p + os.pathsep + os.environ.get("PATH", "")

# Now check for ffmpeg
ffmpeg_path = shutil.which('ffmpeg')
if ffmpeg_path:
    print(f"ffmpeg found at: {ffmpeg_path}")
    result = subprocess.run(['ffmpeg', '-version'], capture_output=True, text=True)
    # Print just the first line (version info)
    print(result.stdout.split('\n')[0])
else:
    print("WARNING: ffmpeg not found!")
    print("Please install ffmpeg before continuing:")
    if platform.system() == "Darwin":
        print("  Mac: brew install ffmpeg")
    elif platform.system() == "Windows":
        print("  Windows: choco install ffmpeg")
        print("  Or download from https://ffmpeg.org/download.html")
    else:
        print("  Linux: sudo apt install ffmpeg")
    print()
    print("If ffmpeg IS installed but still not found, find its location in")
    print("your terminal with 'which ffmpeg' (Mac/Linux) or 'where ffmpeg' (Windows)")
    print("and add the directory to PATH below:")
    print("  os.environ['PATH'] = '/your/ffmpeg/directory' + os.pathsep + os.environ['PATH']")

In [ ]:
# Import all the libraries we'll need
# If any of these fail, make sure you've run: uv pip install -r requirements.txt
# And also: uv pip install openai-whisper

import whisper
import sounddevice as sd
import soundfile as sf
import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display
import time
import os
import torch

print("All libraries imported successfully!")
print(f"Whisper version: {whisper.__version__}")
print(f"PyTorch version: {torch.__version__}")
print(f"Available Whisper models: {whisper.available_models()}")

# Check for GPU / Apple Silicon acceleration
if torch.cuda.is_available():
    print(f"\nGPU available: {torch.cuda.get_device_name(0)}")
    print("Whisper will use your GPU for faster transcription!")
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    print("\nApple Silicon (MPS) detected!")
    print("Note: Whisper will primarily use CPU, but PyTorch has MPS support.")
else:
    print("\nNo GPU detected - using CPU")
    print("This is totally fine! The 'tiny' and 'base' models run great on CPU.")

## Choosing a Whisper Model

Whisper comes in several sizes. Smaller models are faster but less accurate. For working locally on your laptop, **tiny** or **base** are great starting points!

| Model | Parameters | Download Size | Relative Speed | Best For |
|-------|-----------|---------------|----------------|----------|
| `tiny` | 39M | ~75 MB | ~10x | Quick experiments, real-time on CPU |
| `base` | 74M | ~140 MB | ~7x | Good balance for local use |
| `small` | 244M | ~460 MB | ~4x | Better accuracy, still comfortable on M1 |
| `medium` | 769M | ~1.5 GB | ~2x | High accuracy, needs decent hardware |
| `large` | 1550M | ~3 GB | 1x | Best accuracy, GPU recommended |

**For this class**: Start with `tiny` to get things working fast, then try `base` or `small` if you want better results.

**Pro tip**: On Apple Silicon Macs (M1/M2/M3), the `small` model runs very comfortably!

In [ ]:
# Load a Whisper model
# The first time you run this, it will DOWNLOAD the model (~75MB for tiny)
# After that, it's cached locally on your machine - no internet needed!

model_name = "tiny"  # Start with tiny - change to "base" or "small" later!

print(f"Loading Whisper '{model_name}' model...")
print("(First time will download - this only happens once!)\n")

model = whisper.load_model(model_name)

print(f"Model '{model_name}' loaded successfully!")
print(f"Model has {sum(p.numel() for p in model.parameters()):,} parameters")
print(f"\nFor comparison, our Audrey CNN had ~{128 * 16 * 45 + 512 * 10:,} parameters")
print(f"Whisper '{model_name}' is about {sum(p.numel() for p in model.parameters()) // 100000}x bigger!")

# Part 3: Your First Transcription

Let's record some audio and transcribe it with Whisper! This is the moment where months of AI research become a single function call.

**Important**: Whisper works best with audio at **16,000 Hz** (16kHz). This is lower than the 44,100 Hz we used in previous classes, but speech doesn't need those high frequencies - human speech is mostly below 8,000 Hz.

In [ ]:
# Record audio from your microphone
# Say something clearly - a sentence or two works great!

SAMPLE_RATE = 16000  # 16kHz - what Whisper expects
DURATION = 5         # Record for 5 seconds
RECORDING_FILE = "my_recording.wav"

print(f"Recording for {DURATION} seconds at {SAMPLE_RATE} Hz...")
print("Speak now!\n")

recording = sd.rec(
    int(DURATION * SAMPLE_RATE),
    samplerate=SAMPLE_RATE,
    channels=1,
    dtype='float32'
)
sd.wait()

# Save the recording
sf.write(RECORDING_FILE, recording, SAMPLE_RATE)
print(f"Recording saved to '{RECORDING_FILE}'")

# Play it back in the notebook
from IPython.display import Audio, display
display(Audio(RECORDING_FILE))

In [ ]:
# Transcribe the recording with Whisper!
# This is where the magic happens - one line of code!

result = model.transcribe(RECORDING_FILE)

print("=" * 50)
print("TRANSCRIPTION RESULT")
print("=" * 50)
print(f"\nYou said: \"{result['text'].strip()}\"")
print(f"\nDetected language: {result['language']}")

## Understanding the Result Object

When Whisper transcribes audio, it returns a Python dictionary with several useful pieces of information. Let's explore what's inside!

In [ ]:
# The result is a Python dictionary
# Let's look at everything it contains

print("Keys in the result dictionary:")
for key in result.keys():
    print(f"  - {key}")

print("\n" + "=" * 50)
print("DETAILED BREAKDOWN")
print("=" * 50)

# 1. The full transcribed text
print(f"\n1. TEXT (the transcription):")
print(f"   '{result['text']}'")

# 2. The detected language
print(f"\n2. LANGUAGE: '{result['language']}'")

# 3. Segments - Whisper breaks the transcription into time-stamped chunks
print(f"\n3. SEGMENTS ({len(result['segments'])} segments):")
for seg in result['segments']:
    start = seg['start']
    end = seg['end']
    text = seg['text']
    print(f"   [{start:5.1f}s -> {end:5.1f}s] {text}")

print("\nEach segment also contains:")
if result['segments']:
    seg = result['segments'][0]
    for key in seg.keys():
        print(f"  - {key}")

### What Just Happened?

In one function call, Whisper:

1. Loaded your audio file
2. Converted it to a mel spectrogram (like we did manually in Class 1!)
3. Passed the spectrogram through its encoder neural network
4. Used its decoder to generate text, token by token
5. Detected the language automatically
6. Split the text into timestamped segments

Compare that to our Audrey model which could only recognize single digits! This is the power of training on 680,000 hours of data with a much larger neural network.

# Part 4: Exploring Whisper's Features

Whisper isn't just a simple speech-to-text system. It has several powerful features built in. Let's explore them!

## Language Detection

Whisper can automatically detect what language is being spoken. This works by analyzing the first 30 seconds of audio and comparing it against all 99+ languages it knows.

Let's see how confident Whisper is about the language of our recording!

In [ ]:
# Language detection - what language does Whisper think you're speaking?

# Step 1: Load and prepare audio in Whisper's expected format
audio = whisper.load_audio(RECORDING_FILE)
audio = whisper.pad_or_trim(audio)  # Whisper expects exactly 30 seconds of audio

# Step 2: Create a mel spectrogram (same concept from Class 1!)
mel = whisper.log_mel_spectrogram(audio, n_mels=model.dims.n_mels).to(model.device)

# Step 3: Detect the language
_, probs = model.detect_language(mel)

# Show the top 10 detected languages with a visual bar
print("Language Detection Results:")
print("=" * 50)
sorted_probs = sorted(probs.items(), key=lambda x: x[1], reverse=True)
for lang, prob in sorted_probs[:10]:
    bar = "*" * int(prob * 50)
    print(f"  {lang}: {prob:6.1%} {bar}")

print(f"\nMost likely language: {sorted_probs[0][0]} ({sorted_probs[0][1]:.1%})")

## Translation

One of Whisper's coolest features: it can **translate** speech from any of its 99+ supported languages directly into English!

If you speak another language, try recording yourself and see how Whisper translates it. If not, try finding a short audio clip in another language online (YouTube, a language learning app, etc.) and save it as a `.wav` or `.mp3` file.

In [ ]:
# Translation demo - Whisper can translate speech to English!
# Try speaking in another language, or use any non-English audio file

TRANSLATION_FILE = "translation_test.wav"

print(f"Recording for {DURATION} seconds...")
print("Try speaking in another language! (or English is fine too)\n")

recording = sd.rec(
    int(DURATION * SAMPLE_RATE),
    samplerate=SAMPLE_RATE,
    channels=1,
    dtype='float32'
)
sd.wait()
sf.write(TRANSLATION_FILE, recording, SAMPLE_RATE)

# Transcribe in the original language
result_original = model.transcribe(TRANSLATION_FILE)
print(f"Transcription (original): {result_original['text']}")
print(f"Detected language: {result_original['language']}")

# Translate to English (using task='translate')
result_translated = model.transcribe(TRANSLATION_FILE, task="translate")
print(f"\nTranslation (English): {result_translated['text']}")

display(Audio(TRANSLATION_FILE))

## Word-Level Timestamps

Whisper can provide timestamps for each individual word in the transcription. This is incredibly useful for:

- **Subtitling** videos
- **Karaoke** applications
- **Aligning** text with audio
- **Creative coding** projects (trigger visuals at the exact moment specific words are spoken!)

In [ ]:
# Get word-level timestamps
# word_timestamps=True gives us the exact timing for each word

result = model.transcribe(
    RECORDING_FILE,
    word_timestamps=True
)

print("Word-Level Timestamps:")
print("=" * 50)

# Each segment contains a list of words with their timestamps
for segment in result['segments']:
    if 'words' in segment:
        for word_info in segment['words']:
            word = word_info['word']
            start = word_info['start']
            end = word_info['end']
            print(f"  [{start:6.2f}s -> {end:6.2f}s]  {word}")

In [ ]:
# Visualize the word timestamps on top of the waveform!
# This shows exactly when each word appears in the audio

# Load audio for visualization
y, sr = librosa.load(RECORDING_FILE, sr=None)
duration = len(y) / sr

# Create the plot
fig, ax = plt.subplots(figsize=(14, 4))

# Plot waveform
times = np.linspace(0, duration, len(y))
ax.plot(times, y, alpha=0.5, color='steelblue')

# Collect all words with timestamps
all_words = []
for segment in result['segments']:
    if 'words' in segment:
        for word_info in segment['words']:
            all_words.append(word_info)

# Add colored regions for each word
if all_words:
    colors = plt.cm.Set2(np.linspace(0, 1, max(len(all_words), 1)))
    for i, word_info in enumerate(all_words):
        start = word_info['start']
        end = word_info['end']
        word = word_info['word'].strip()

        # Highlight the word region
        ax.axvspan(start, end, alpha=0.2, color=colors[i])

        # Add the word label
        mid = (start + end) / 2
        ax.text(mid, ax.get_ylim()[1] * 0.85, word,
                ha='center', fontsize=8, fontweight='bold')

ax.set_xlabel('Time (seconds)')
ax.set_ylabel('Amplitude')
ax.set_title('Waveform with Word-Level Timestamps')
ax.set_xlim(0, duration)
plt.tight_layout()
plt.show()

## Comparing Model Sizes

How much difference does model size make? Let's transcribe the same audio with different models and compare speed and accuracy.

**Note**: Each model downloads on first use. The `tiny` model is ~75MB, `base` is ~140MB.

In [ ]:
# Compare different model sizes on the same audio
# This will download each model the first time (only happens once!)

models_to_compare = ["tiny", "base"]  # Add "small" if you have time and disk space!

print(f"Comparing models on: {RECORDING_FILE}")
print("=" * 60)

results = []

for name in models_to_compare:
    print(f"\nLoading '{name}' model...")
    test_model = whisper.load_model(name)

    start_time = time.time()
    test_result = test_model.transcribe(RECORDING_FILE)
    elapsed = time.time() - start_time

    results.append({
        'model': name,
        'time': elapsed,
        'text': test_result['text'].strip(),
        'language': test_result['language']
    })

    print(f"  Model:    {name}")
    print(f"  Time:     {elapsed:.2f} seconds")
    print(f"  Text:     {test_result['text'].strip()}")
    print(f"  Language: {test_result['language']}")

    # Free memory
    del test_model

print("\n" + "=" * 60)
print("Observations:")
print("- Larger models are more accurate but slower")
print("- For real-time use, 'tiny' or 'base' is recommended")
print("- For best quality (offline), 'small' or 'medium' is great")

# Reload our working model
model = whisper.load_model("tiny")

# Part 5: Working with Longer Audio

Whisper can handle audio of any length! Under the hood, it splits longer audio into **30-second chunks** and processes them one at a time. This means you can transcribe podcasts, lectures, interviews - anything!

Let's try a longer recording.

In [ ]:
# Record a longer clip
# Try reading a paragraph, telling a story, or just talking for 15 seconds!

LONG_DURATION = 15  # seconds
LONG_RECORDING_FILE = "long_recording.wav"

print(f"Recording for {LONG_DURATION} seconds...")
print("Try reading a paragraph or telling a short story!\n")

recording = sd.rec(
    int(LONG_DURATION * SAMPLE_RATE),
    samplerate=SAMPLE_RATE,
    channels=1,
    dtype='float32'
)
sd.wait()
sf.write(LONG_RECORDING_FILE, recording, SAMPLE_RATE)
print(f"Recording saved!\n")

# Transcribe with word timestamps
result = model.transcribe(LONG_RECORDING_FILE, word_timestamps=True)

print("=" * 50)
print("TRANSCRIPTION:")
print(result['text'])

print("\n\nSEGMENTS (with timestamps):")
for seg in result['segments']:
    print(f"  [{seg['start']:6.1f}s -> {seg['end']:6.1f}s] {seg['text']}")

# Play it back
display(Audio(LONG_RECORDING_FILE))

In [ ]:
# Visualize the longer transcription as a spectrogram with segment labels

y, sr = librosa.load(LONG_RECORDING_FILE, sr=None)

fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

# Top: Waveform with segment labels
duration = len(y) / sr
times = np.linspace(0, duration, len(y))
axes[0].plot(times, y, alpha=0.6, color='steelblue')
axes[0].set_ylabel('Amplitude')
axes[0].set_title('Waveform with Transcription Segments')

# Add segment labels
colors = plt.cm.Pastel1(np.linspace(0, 1, max(len(result['segments']), 1)))
for i, seg in enumerate(result['segments']):
    axes[0].axvspan(seg['start'], seg['end'], alpha=0.3, color=colors[i % len(colors)])
    mid = (seg['start'] + seg['end']) / 2
    # Truncate long text for the label
    label = seg['text'].strip()[:30] + ('...' if len(seg['text'].strip()) > 30 else '')
    axes[0].text(mid, axes[0].get_ylim()[1] * 0.8, label,
                 ha='center', fontsize=7, style='italic')

# Bottom: Mel spectrogram
S = librosa.feature.melspectrogram(y=y, sr=sr)
S_db = librosa.power_to_db(S, ref=np.max)
librosa.display.specshow(S_db, x_axis='time', y_axis='mel', sr=sr, ax=axes[1])
axes[1].set_title('Mel Spectrogram')

plt.tight_layout()
plt.show()

# Part 6: Real-Time Speech Recognition

Now for the exciting part - **real-time** speech recognition! We'll set up Whisper to listen to your microphone and transcribe what you say.

### How Real-Time Works with Whisper

Whisper is **not** a streaming model - it processes complete audio chunks. For "real-time" recognition, we use a **record-then-transcribe** approach:

1. Record a short chunk of audio (e.g., 3-5 seconds)
2. Send it to Whisper for transcription
3. Display the result
4. Repeat!

This introduces a small delay (the time it takes to transcribe each chunk), but with the `tiny` model this is usually under 1 second.

### Two Approaches

| Approach | How it works | Best for |
|----------|-------------|----------|
| **Enter-to-Record** | Press Enter, speak, get result | Simple and reliable |
| **Continuous Listening** | Auto-detects when you speak | Hands-free, more interactive |

In [ ]:
# =============================================================================
# APPROACH 1: Press Enter to Record and Transcribe
# =============================================================================
# Simple and reliable - press Enter, speak, get transcription!

RECORD_DURATION = 5  # seconds per recording

print("=" * 50)
print("PRESS-TO-RECORD SPEECH RECOGNITION")
print("=" * 50)
print(f"How it works:")
print(f"  1. Press Enter when ready")
print(f"  2. Speak for up to {RECORD_DURATION} seconds")
print(f"  3. See the transcription!")
print(f"  4. Type 'q' to quit")
print("=" * 50)

try:
    while True:
        user_input = input("\nPress Enter to record (or 'q' to quit): ")
        if user_input.lower() == 'q':
            break

        # Record
        print(f"Recording for {RECORD_DURATION} seconds... Speak now!")
        recording = sd.rec(
            int(RECORD_DURATION * SAMPLE_RATE),
            samplerate=SAMPLE_RATE,
            channels=1,
            dtype='float32'
        )
        sd.wait()

        # Save to a temporary file
        temp_file = "_temp_realtime.wav"
        sf.write(temp_file, recording, SAMPLE_RATE)

        # Transcribe
        start_time = time.time()
        result = model.transcribe(temp_file)
        elapsed = time.time() - start_time

        print(f"\n  You said: \"{result['text'].strip()}\"")
        print(f"  Language: {result['language']}")
        print(f"  Transcription took: {elapsed:.2f}s")

        # Clean up
        if os.path.exists(temp_file):
            os.remove(temp_file)

except KeyboardInterrupt:
    pass
finally:
    if os.path.exists("_temp_realtime.wav"):
        os.remove("_temp_realtime.wav")
    print("\nStopped!")

In [ ]:
# =============================================================================
# APPROACH 2: Continuous Listening with Automatic Speech Detection
# =============================================================================
# Automatically detects when you start speaking and transcribes!
# Uses a rolling buffer to capture the BEGINNING of your speech.

from collections import deque
from IPython.display import clear_output

# Audio settings
SAMPLE_RATE = 16000
BLOCK_SIZE = 1024

# Speech detection settings - ADJUST THESE FOR YOUR ENVIRONMENT
# If it triggers on background noise: increase VOLUME_THRESHOLD (try 0.05 or 0.08)
# If it misses your speech: decrease VOLUME_THRESHOLD (try 0.02)
VOLUME_THRESHOLD = 0.03
SILENCE_DURATION = 1.5    # Seconds of silence before we process
BUFFER_SECONDS = 0.5      # Pre-speech buffer (captures the start of words!)
MAX_RECORDING = 10        # Maximum recording length (seconds)
MIN_RECORDING = 0.5       # Minimum recording to process (seconds)

# Calculate sizes in blocks/samples
silence_blocks = int(SILENCE_DURATION * SAMPLE_RATE / BLOCK_SIZE)
buffer_blocks = int(BUFFER_SECONDS * SAMPLE_RATE / BLOCK_SIZE)
max_samples = int(MAX_RECORDING * SAMPLE_RATE)
min_samples = int(MIN_RECORDING * SAMPLE_RATE)

# State variables
rolling_buffer = deque(maxlen=buffer_blocks)
is_recording = False
recorded_audio = []
silence_counter = 0
TEMP_FILE = "_temp_continuous.wav"

def audio_callback(indata, frames, time_info, status):
    """Called for each block of audio from the microphone."""
    global is_recording, recorded_audio, silence_counter, rolling_buffer

    audio_block = indata[:, 0].copy()
    rms = np.sqrt(np.mean(audio_block**2))

    if not is_recording:
        # Keep audio in rolling buffer (this captures word beginnings!)
        rolling_buffer.append(audio_block)

        # Check for speech onset
        if rms > VOLUME_THRESHOLD:
            is_recording = True
            silence_counter = 0
            # Include the buffer contents!
            recorded_audio = list(np.concatenate(list(rolling_buffer)))
            recorded_audio.extend(audio_block.tolist())
            print("\r  [Recording...] Speak your sentence!", end="", flush=True)
    else:
        # Currently recording
        recorded_audio.extend(audio_block.tolist())

        if rms < VOLUME_THRESHOLD:
            silence_counter += 1
        else:
            silence_counter = 0

        # Stop recording after silence or max duration
        total_samples = len(recorded_audio)
        if (silence_counter >= silence_blocks and total_samples >= min_samples) or total_samples >= max_samples:
            audio_data = np.array(recorded_audio, dtype=np.float32)

            # Save and transcribe
            sf.write(TEMP_FILE, audio_data, SAMPLE_RATE)

            try:
                result = model.transcribe(TEMP_FILE)
                text = result['text'].strip()

                if text:
                    clear_output(wait=True)
                    print("=" * 50)
                    print("CONTINUOUS SPEECH RECOGNITION")
                    print("=" * 50)
                    print(f"\n  You said: \"{text}\"")
                    print(f"  Language: {result['language']}")
                    duration = len(audio_data) / SAMPLE_RATE
                    print(f"  Duration: {duration:.1f}s")
                    print(f"\nListening... (speak to transcribe)")
                    print("Press the STOP button or Kernel > Interrupt to stop")
            except Exception as e:
                print(f"\n  Error: {e}")

            # Reset state
            is_recording = False
            recorded_audio = []
            silence_counter = 0
            rolling_buffer.clear()

# Start listening
print("=" * 50)
print("CONTINUOUS SPEECH RECOGNITION")
print("=" * 50)
print(f"Volume threshold: {VOLUME_THRESHOLD} RMS")
print(f"Silence timeout:  {SILENCE_DURATION}s")
print(f"Max recording:    {MAX_RECORDING}s")
print("=" * 50)
print("\nListening... (speak to transcribe)")
print("Press the STOP button or Kernel > Interrupt to stop")

stream = None
try:
    stream = sd.InputStream(
        samplerate=SAMPLE_RATE,
        blocksize=BLOCK_SIZE,
        channels=1,
        callback=audio_callback
    )
    stream.start()

    while True:
        sd.sleep(100)

except KeyboardInterrupt:
    pass
finally:
    if stream is not None:
        stream.stop()
        stream.close()
    if os.path.exists(TEMP_FILE):
        os.remove(TEMP_FILE)
    rolling_buffer.clear()
    is_recording = False
    recorded_audio = []
    silence_counter = 0
    print("\nStopped!")

# Part 7: Triggering Actions with Speech

Now that we have real-time transcription, we can use the **text** Whisper outputs to trigger actions in other programs! This is where speech recognition becomes truly interactive.

You've already done this in previous classes:
- **Class 1**: Pitch detection -> Arduino LED
- **Class 2**: Digit recognition -> Actions

Now instead of detecting *pitch* or *digits*, we can detect **any word or phrase**!

### Ideas for Speech-Triggered Actions

| Action | How | Example |
|--------|-----|--------|
| Print to console | Built-in Python | "Turn on" -> prints "Activating!" |
| Send OSC message | `python-osc` library | "Red" -> sends color to p5.js |
| Send Serial | `pyserial` library | "Light on" -> turns on Arduino LED |
| Write to file | Built-in Python | Transcribe a meeting -> save notes |
| Play a sound | `sounddevice` | "Beep" -> plays a tone |

In [ ]:
# =============================================================================
# SPEECH COMMAND DETECTOR
# =============================================================================
# Records audio, transcribes it, and checks for keywords to trigger actions!
# This is the foundation for voice-controlled projects.

def on_command(command, full_text):
    """
    This function is called when a keyword is detected.
    CUSTOMIZE THIS to trigger whatever you want!

    Ideas:
    - Send OSC to p5.js
    - Send serial to Arduino
    - Play a sound
    - Change a variable
    - Write to a file
    """
    print(f"  >>> COMMAND DETECTED: '{command}' <<<")
    print(f"  >>> Full text: '{full_text}' <<<")


# Define your keywords/commands - add whatever words you want to detect!
COMMANDS = {
    "hello": "greeting",
    "goodbye": "farewell",
    "red": "color_red",
    "blue": "color_blue",
    "green": "color_green",
    "on": "activate",
    "off": "deactivate",
    "yes": "affirm",
    "no": "deny",
}

RECORD_DURATION = 3  # seconds

print("=" * 50)
print("SPEECH COMMAND DETECTOR")
print("=" * 50)
print(f"Listening for keywords: {list(COMMANDS.keys())}")
print(f"Press Enter to record ({RECORD_DURATION}s), 'q' to quit")
print("=" * 50)

try:
    while True:
        user_input = input("\nPress Enter to record: ")
        if user_input.lower() == 'q':
            break

        # Record
        print(f"Recording for {RECORD_DURATION}s... Say a command!")
        recording = sd.rec(
            int(RECORD_DURATION * SAMPLE_RATE),
            samplerate=SAMPLE_RATE,
            channels=1,
            dtype='float32'
        )
        sd.wait()

        # Save and transcribe
        temp_file = "_temp_command.wav"
        sf.write(temp_file, recording, SAMPLE_RATE)
        result = model.transcribe(temp_file)
        text = result['text'].strip().lower()

        print(f"\n  Heard: \"{text}\"")

        # Check for commands
        found_command = False
        for keyword, action in COMMANDS.items():
            if keyword in text:
                on_command(keyword, text)
                found_command = True

        if not found_command:
            print("  (No command keyword detected)")

        # Clean up
        if os.path.exists(temp_file):
            os.remove(temp_file)

except KeyboardInterrupt:
    print("\nStopped!")

## Example: Sending OSC Messages to p5.js

You've already seen how to send OSC messages in previous classes. Here's how to combine speech recognition with OSC to control a p5.js sketch (or any OSC receiver) with your voice!

**Install python-osc first:**
```bash
uv pip install python-osc
```

In [ ]:
# =============================================================================
# SPEECH -> OSC: Control p5.js (or any OSC receiver) with your voice!
# =============================================================================
# This sends OSC messages based on what Whisper hears.
# You'll need a p5.js sketch or other program listening for OSC messages.

# Uncomment the lines below when you're ready to use OSC:
# from pythonosc import udp_client
# OSC_IP = "127.0.0.1"   # localhost (same computer)
# OSC_PORT = 12000        # match this to your p5.js sketch
# osc_client = udp_client.SimpleUDPClient(OSC_IP, OSC_PORT)

def send_osc_from_speech(text):
    """
    Analyze transcribed text and send OSC messages.
    Customize this for your project!
    """
    text = text.strip().lower()

    # Example 1: Send the full transcription as a string
    # osc_client.send_message("/speech/text", text)
    print(f"  [OSC] /speech/text -> '{text}'")

    # Example 2: Detect colors and send RGB values
    colors = {
        "red": [255, 0, 0],
        "blue": [0, 0, 255],
        "green": [0, 255, 0],
        "yellow": [255, 255, 0],
        "purple": [128, 0, 128],
        "white": [255, 255, 255],
        "black": [0, 0, 0],
    }

    for color_name, rgb in colors.items():
        if color_name in text:
            # osc_client.send_message("/speech/color", rgb)
            print(f"  [OSC] /speech/color -> {rgb} ({color_name})")

    # Example 3: Detect numbers and send them
    number_words = {
        "one": 1, "two": 2, "three": 3, "four": 4, "five": 5,
        "six": 6, "seven": 7, "eight": 8, "nine": 9, "ten": 10,
    }

    for word, num in number_words.items():
        if word in text:
            # osc_client.send_message("/speech/number", num)
            print(f"  [OSC] /speech/number -> {num}")


# Quick test with some example sentences
print("Testing OSC speech commands:")
print("=" * 50)

test_sentences = [
    "Make it red",
    "Change to blue please",
    "Set the number to five",
    "Hello world",
]

for sentence in test_sentences:
    print(f"\nInput: '{sentence}'")
    send_osc_from_speech(sentence)

print("\n" + "=" * 50)
print("To use with real speech, combine with the")
print("real-time recording code from Part 6!")

## Example: Controlling Arduino with Speech

Just like we controlled the LED with pitch in Class 1, we can now control it with **words**!

Say "on" or "light" to turn the LED on, "off" or "dark" to turn it off.

In [ ]:
# =============================================================================
# SPEECH -> SERIAL: Control Arduino with your voice!
# =============================================================================
# Says "on" or "light" -> sends '1' to Arduino (LED on)
# Says "off" or "dark" -> sends '0' to Arduino (LED off)

# Uncomment and configure when you have Arduino connected:
# import serial
# SERIAL_PORT = '/dev/cu.usbmodem...'  # Find yours: ls /dev/tty* | grep usb
# BAUD_RATE = 9600
# arduino = serial.Serial(SERIAL_PORT, BAUD_RATE)
# time.sleep(2)  # Wait for Arduino reset

def send_serial_from_speech(text):
    """
    Analyze transcribed text and send serial commands to Arduino.
    Customize this for your project!
    """
    text = text.strip().lower()

    if "on" in text or "light" in text:
        # arduino.write(b'1')
        print(f"  [Serial] Sending '1' (LED ON)")
    elif "off" in text or "dark" in text:
        # arduino.write(b'0')
        print(f"  [Serial] Sending '0' (LED OFF)")
    else:
        print(f"  [Serial] No command recognized in: '{text}'")


# Quick test
print("Testing Serial speech commands:")
print("=" * 50)

test_sentences = ["Turn on the light", "Turn it off", "Hello world"]
for sentence in test_sentences:
    print(f"\nInput: '{sentence}'")
    send_serial_from_speech(sentence)

print("\n" + "=" * 50)
print("Combine with real-time recording from Part 6")
print("to control Arduino with your voice!")

In [ ]:
# =============================================================================
# PUTTING IT ALL TOGETHER: Live Speech -> Action
# =============================================================================
# This combines real-time recording with action triggering.
# Customize the on_speech() function to do whatever you want!

def on_speech(text):
    """
    Called every time Whisper transcribes something.
    This is YOUR function to customize!

    The 'text' parameter is the transcribed speech as a string.
    You can:
      - Check for keywords
      - Send OSC messages
      - Send serial to Arduino
      - Play sounds
      - Write to a file
      - Anything you can do with Python!
    """
    text_lower = text.strip().lower()
    print(f"  Transcription: \"{text}\"")

    # === ADD YOUR CUSTOM ACTIONS HERE ===

    # Example: keyword detection
    if "hello" in text_lower:
        print("  -> Hello to you too!")
    if "red" in text_lower:
        print("  -> Color: RED detected!")
        # osc_client.send_message("/color", [255, 0, 0])
    if "stop" in text_lower:
        print("  -> STOP command received!")


RECORD_DURATION = 4  # seconds per recording

print("=" * 50)
print("LIVE SPEECH -> ACTION")
print("=" * 50)
print("Speak and watch your custom actions trigger!")
print("Press Enter to record, 'q' to quit")
print("=" * 50)

try:
    while True:
        user_input = input("\nPress Enter to record: ")
        if user_input.lower() == 'q':
            break

        print(f"Recording {RECORD_DURATION}s... Speak now!")
        recording = sd.rec(
            int(RECORD_DURATION * SAMPLE_RATE),
            samplerate=SAMPLE_RATE,
            channels=1,
            dtype='float32'
        )
        sd.wait()

        temp_file = "_temp_action.wav"
        sf.write(temp_file, recording, SAMPLE_RATE)
        result = model.transcribe(temp_file)

        # Call YOUR custom function!
        on_speech(result['text'])

        if os.path.exists(temp_file):
            os.remove(temp_file)

except KeyboardInterrupt:
    print("\nStopped!")

# Part 8: Fine-Tuning Whisper (Stretch Goal)

**What is fine-tuning?** Taking a pre-trained model (like Whisper) and training it further on your own specific data to make it better at a particular task.

Think of it like this: Whisper already knows how to understand speech in general. Fine-tuning teaches it to be *extra good* at understanding a specific type of speech.

### Why Fine-Tune?

| Use Case | Example |
|----------|---------|
| **Specialized vocabulary** | Medical terms, legal jargon, technical language |
| **Specific accents** | Regional dialects, non-native speakers |
| **Low-resource languages** | Languages with limited Whisper training data |
| **Noisy environments** | Factories, concerts, underwater |

### Requirements

Fine-tuning is **computationally expensive**. You'll need:
- An **NVIDIA GPU** with CUDA support, OR
- **Google Colab** (free GPU access!)
- The `transformers` and `datasets` libraries from HuggingFace

### Finding Training Datasets

HuggingFace has thousands of speech datasets you can use:

- [Common Voice](https://huggingface.co/datasets/mozilla-foundation/common_voice_16_1) - Mozilla's crowdsourced speech dataset (100+ languages)
- [LibriSpeech](https://huggingface.co/datasets/librispeech_asr) - English audiobook recordings
- [FLEURS](https://huggingface.co/datasets/google/fleurs) - Google's multilingual speech benchmark
- [Browse all ASR datasets](https://huggingface.co/datasets?task_categories=automatic-speech-recognition)

### Running on Google Colab

1. Go to [colab.research.google.com](https://colab.research.google.com)
2. Create a new notebook
3. Go to **Runtime -> Change runtime type -> GPU (T4)**
4. Copy the code from the cell below into Colab
5. Uncomment everything and run!

**Detailed guide**: [HuggingFace Fine-Tune Whisper Blog Post](https://huggingface.co/blog/fine-tune-whisper)

In [ ]:
# =============================================================================
# FINE-TUNING WHISPER (GPU / GOOGLE COLAB RECOMMENDED)
# =============================================================================
# This code shows how to fine-tune Whisper on a custom dataset
# using HuggingFace Transformers.
#
# To run this:
#   1. Go to colab.research.google.com
#   2. Runtime -> Change runtime type -> GPU (T4)
#   3. Install: !pip install transformers datasets evaluate jiwer accelerate
#   4. Uncomment the code below and run!

# --- Uncomment everything below to run on Google Colab ---

# from transformers import WhisperForConditionalGeneration, WhisperProcessor
# from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
# from datasets import load_dataset, Audio
# import torch

# # Step 1: Load the pre-trained Whisper model and processor
# model_name = "openai/whisper-tiny"  # Start small for faster experiments!
# processor = WhisperProcessor.from_pretrained(model_name)
# model = WhisperForConditionalGeneration.from_pretrained(model_name)

# # Step 2: Load a dataset from HuggingFace
# # This example uses Common Voice for Hindi - change the language code!
# dataset = load_dataset(
#     "mozilla-foundation/common_voice_16_1",
#     "hi",  # Language code: "hi"=Hindi, "es"=Spanish, "fr"=French, etc.
#     split="train[:100]",  # Start with a small subset for testing
#     trust_remote_code=True,
# )

# # Step 3: Preprocess the audio data
# def prepare_dataset(batch):
#     audio = batch["audio"]
#     # Convert audio to the format Whisper expects
#     input_features = processor(
#         audio["array"],
#         sampling_rate=audio["sampling_rate"],
#         return_tensors="pt"
#     ).input_features[0]
#     # Encode the target text
#     batch["input_features"] = input_features
#     batch["labels"] = processor.tokenizer(batch["sentence"]).input_ids
#     return batch

# # Resample all audio to 16kHz (what Whisper expects)
# dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))
# dataset = dataset.map(prepare_dataset, remove_columns=dataset.column_names)

# # Step 4: Set up training
# training_args = Seq2SeqTrainingArguments(
#     output_dir="./whisper-finetuned",
#     per_device_train_batch_size=8,
#     learning_rate=1e-5,
#     num_train_epochs=3,
#     fp16=True,  # Mixed precision (faster on GPU)
#     logging_steps=10,
#     save_steps=100,
#     predict_with_generate=True,
# )

# # Step 5: Create trainer and fine-tune!
# trainer = Seq2SeqTrainer(
#     model=model,
#     args=training_args,
#     train_dataset=dataset,
#     tokenizer=processor.feature_extractor,
# )

# # This starts fine-tuning! It will take a few minutes on a T4 GPU.
# # trainer.train()

# # Step 6: Save the fine-tuned model
# # model.save_pretrained("./whisper-finetuned-final")
# # processor.save_pretrained("./whisper-finetuned-final")

print("=" * 60)
print("FINE-TUNING WHISPER - STRETCH GOAL")
print("=" * 60)
print()
print("This code is designed to run on Google Colab with a GPU.")
print()
print("To try it:")
print("  1. Go to colab.research.google.com")
print("  2. Create a new notebook")
print("  3. Runtime -> Change runtime type -> GPU (T4)")
print("  4. !pip install transformers datasets evaluate jiwer accelerate")
print("  5. Copy this cell's code, uncomment everything, and run!")
print()
print("Resources:")
print("  - HuggingFace guide: https://huggingface.co/blog/fine-tune-whisper")
print("  - Google Colab: https://colab.research.google.com")
print("  - Datasets: https://huggingface.co/datasets?task_categories=automatic-speech-recognition")

# Part 9: Exploring Other ASR Systems

Whisper is amazing, but it's just one of many ASR systems available today. Here's a tour of what's out there!

## Open Source (Run Locally - No API Key Needed)

| System | By | Highlights |
|--------|----|------------|
| [**Whisper**](https://github.com/openai/whisper) | OpenAI | 99+ languages, translation, timestamps |
| [**Faster Whisper**](https://github.com/SYSTRAN/faster-whisper) | SYSTRAN | Same as Whisper but 4x faster! Uses CTranslate2 |
| [**Distil-Whisper**](https://github.com/huggingface/distil-whisper) | HuggingFace | 6x faster than Whisper, nearly same accuracy |
| [**Wav2Vec 2.0**](https://github.com/facebookresearch/fairseq) | Meta | Self-supervised learning, great for low-resource languages |
| [**NeMo Canary/Parakeet**](https://github.com/NVIDIA/NeMo) | NVIDIA | State-of-the-art English ASR, extremely accurate |
| [**Vosk**](https://alphacephei.com/vosk/) | Alpha Cephei | Lightweight, offline, runs on Raspberry Pi! |

## Cloud APIs (Requires Internet + API Key)

| Service | By | Highlights |
|---------|----|------------|
| [**AssemblyAI**](https://www.assemblyai.com) | AssemblyAI | Excellent accuracy, speaker diarization, sentiment |
| [**Deepgram**](https://deepgram.com) | Deepgram | Ultra-fast, real-time streaming, custom models |
| [**Voxtral**](https://mistral.ai) | Mistral | Multimodal model - understands speech AND context |
| [**Google Cloud Speech**](https://cloud.google.com/speech-to-text) | Google | 125+ languages, streaming, specialized models |
| [**Azure Speech**](https://azure.microsoft.com/en-us/products/ai-services/speech-to-text) | Microsoft | Real-time, batch, custom pronunciation |
| [**Amazon Transcribe**](https://aws.amazon.com/transcribe/) | Amazon | Real-time, medical transcription, call analytics |
| [**Speechmatics**](https://www.speechmatics.com) | Speechmatics | 50+ languages, excellent accuracy |

## AssemblyAI

[AssemblyAI](https://www.assemblyai.com) is one of the most popular cloud ASR APIs. Beyond basic transcription, it offers:
- **Speaker diarization** (who said what)
- **Sentiment analysis** (positive/negative/neutral)
- **Topic detection**
- **Content moderation**
- Free tier available!

```bash
uv pip install assemblyai
```

In [ ]:
# =============================================================================
# ASSEMBLYAI EXAMPLE (Requires free API Key)
# =============================================================================
# Sign up at https://www.assemblyai.com for a free API key
# Then uncomment the code below!

# import assemblyai as aai

# # Set your API key
# aai.settings.api_key = "YOUR_API_KEY_HERE"

# # Create a transcriber
# transcriber = aai.Transcriber()

# # Transcribe a local audio file
# transcript = transcriber.transcribe(RECORDING_FILE)
# print(f"Text: {transcript.text}")

# # AssemblyAI can also identify different speakers!
# config = aai.TranscriptionConfig(speaker_labels=True)
# transcript = transcriber.transcribe(RECORDING_FILE, config=config)
# if transcript.utterances:
#     for utterance in transcript.utterances:
#         print(f"Speaker {utterance.speaker}: {utterance.text}")

print("AssemblyAI example (uncomment code and add API key to try!)")
print("Sign up for free: https://www.assemblyai.com")

## Deepgram

[Deepgram](https://deepgram.com) is known for **speed** and **real-time streaming**. It's great for:
- Ultra-low latency transcription
- Live streaming (WebSocket-based)
- Custom model training on your data
- Free tier available!

```bash
uv pip install deepgram-sdk
```

In [ ]:
# =============================================================================
# DEEPGRAM EXAMPLE (Requires free API Key)
# =============================================================================
# Sign up at https://deepgram.com for a free API key
# Then uncomment the code below!

# from deepgram import DeepgramClient, PrerecordedOptions

# DEEPGRAM_API_KEY = "YOUR_API_KEY_HERE"
# deepgram = DeepgramClient(DEEPGRAM_API_KEY)

# # Read audio file
# with open(RECORDING_FILE, "rb") as audio_file:
#     buffer_data = audio_file.read()

# # Configure transcription
# options = PrerecordedOptions(
#     model="nova-3",         # Deepgram's latest model
#     smart_format=True,      # Auto-punctuation and formatting
#     language="en",
# )

# # Transcribe
# payload = {"buffer": buffer_data}
# response = deepgram.listen.rest.v("1").transcribe_file(payload, options)
# transcript = response.results.channels[0].alternatives[0].transcript
# print(f"Text: {transcript}")

print("Deepgram example (uncomment code and add API key to try!)")
print("Sign up for free: https://deepgram.com")

## More Systems to Explore

### Faster Whisper

If Whisper feels slow on your machine, try [Faster Whisper](https://github.com/SYSTRAN/faster-whisper). It uses the same model weights but runs them through CTranslate2, which is optimized for CPU inference. The result: **~4x faster** with almost identical accuracy!

```bash
uv pip install faster-whisper
```

```python
from faster_whisper import WhisperModel

model = WhisperModel("tiny", device="cpu", compute_type="int8")
segments, info = model.transcribe("audio.wav")

for segment in segments:
    print(f"[{segment.start:.2f}s -> {segment.end:.2f}s] {segment.text}")
```

### Voxtral (Mistral)

[Voxtral](https://mistral.ai) is Mistral's multimodal model that can understand speech. Unlike traditional ASR systems that just convert speech to text, Voxtral can answer *questions* about audio content - "What language is being spoken?" "What is the speaker's mood?" "Summarize this conversation."

### NVIDIA NeMo (Canary / Parakeet)

If you have an NVIDIA GPU, [NeMo](https://github.com/NVIDIA/NeMo) offers some of the most accurate ASR models available, especially for English. The Parakeet models consistently top the [Open ASR Leaderboard](https://huggingface.co/spaces/hf-audio/open_asr_leaderboard).

### Vosk

[Vosk](https://alphacephei.com/vosk/) is perfect for **embedded** and **edge** devices. It runs on Raspberry Pi, Android, iOS, and more with very small model sizes (as small as 50MB). Great for offline applications where you need speech recognition without internet access.

## Quick Reference: Which ASR System Should I Use?

| If you need... | Use this |
|---------------|----------|
| Free, local, multilingual | **Whisper** |
| Fast local transcription | **Faster Whisper** or **Distil-Whisper** |
| Best cloud accuracy | **AssemblyAI** or **Deepgram** |
| Real-time streaming (cloud) | **Deepgram** |
| Speaker identification | **AssemblyAI** or **Google Cloud** |
| Runs on Raspberry Pi | **Vosk** |
| Best English accuracy (GPU) | **NVIDIA NeMo Parakeet** |
| Multimodal understanding | **Voxtral** |
| Fine-tuning on custom data | **Whisper** via HuggingFace |

# Exercises and Homework

In [ ]:
# Exercise 1: Transcribe and Visualize
#
# Record yourself reading a paragraph or poem.
# Use word-level timestamps to create a visualization that highlights
# each word on the waveform as it appears in time.
#
# Hint: Use the word timestamp visualization from Part 4 as a starting point!

In [ ]:
# Exercise 2: Voice-Controlled p5.js
#
# Create a p5.js sketch that responds to voice commands sent over OSC.
# Use the speech -> OSC example from Part 7 as a starting point.
#
# Ideas:
#   - "Circle" / "Square" / "Triangle" -> draw different shapes
#   - Color names -> change the background color
#   - "Bigger" / "Smaller" -> resize elements
#   - "Clear" -> reset the canvas
#   - "Fast" / "Slow" -> change animation speed

In [ ]:
# Exercise 3: Language Quiz
#
# Build a "language detection quiz":
#   1. Record yourself saying a phrase in different languages
#      (use Google Translate to learn phrases if needed!)
#   2. See if Whisper correctly identifies each language
#   3. Track accuracy in a dictionary and display results
#
# Bonus: Try the same phrases with tiny vs base models - which is better?

In [ ]:
# Exercise 4 (Stretch): Real-Time Subtitles
#
# Create a real-time subtitle system that:
#   1. Continuously listens to your microphone
#   2. Transcribes speech in chunks
#   3. Displays a scrolling transcript (like live captions on TV)
#
# Hint: Use the continuous listening code from Part 6 and keep
#       a running list of all transcriptions

In [ ]:
# Exercise 5 (Stretch): Compare ASR Accuracy
#
# Record the same 5 sentences and transcribe them with:
#   - Whisper tiny
#   - Whisper base
#   - Whisper small (if your machine can handle it)
#
# Compare the results:
#   - Which words does each model get wrong?
#   - How much slower are the larger models?
#   - Is the accuracy improvement worth the speed cost?

In [ ]:
# Exercise 6 (Stretch - GPU Required): Fine-Tune Whisper
#
# Follow the fine-tuning guide in Part 8 on Google Colab.
# Pick a language or domain and fine-tune Whisper on a small dataset.
# Compare the fine-tuned model's accuracy to the original!
#
# Resources:
#   - HuggingFace guide: https://huggingface.co/blog/fine-tune-whisper
#   - Google Colab: https://colab.research.google.com

# Extra Credit

- Try [Faster Whisper](https://github.com/SYSTRAN/faster-whisper) and compare its speed to regular Whisper
- Build a voice-controlled Arduino project using full speech recognition (not just pitch!)
- Create a real-time translation demo: speak in one language, display text in English
- Sign up for [AssemblyAI](https://assemblyai.com) or [Deepgram](https://deepgram.com) and compare their results to Whisper
- Explore [HuggingFace's ASR models](https://huggingface.co/models?pipeline_tag=automatic-speech-recognition) - there are hundreds!
- Build a "speech diary" that records and transcribes throughout the day
- Use word-level timestamps to create a karaoke-style lyric display
- Combine Whisper with the emotion recognition from Class 3: detect *what* someone says AND *how* they say it!

---

## Summary

In this notebook you learned:

- **How modern ASR works**: audio -> mel spectrogram -> encoder -> decoder -> text
- **Whisper**: OpenAI's open-source, multilingual ASR model
- **Transcription**: Converting speech to text with a single function call
- **Features**: Language detection, translation, word-level timestamps
- **Model sizes**: Trading off speed vs accuracy (tiny -> large)
- **Real-time recognition**: Record-then-transcribe approach for live speech
- **Triggering actions**: Using transcribed text to control other systems (OSC, Serial)
- **Fine-tuning**: Customizing Whisper for specific tasks (stretch goal)
- **The ASR landscape**: Whisper, Faster Whisper, AssemblyAI, Deepgram, and more

### What's Next?

Now that you can convert speech to text, the possibilities are endless. You can build voice-controlled art installations, real-time captioning systems, multilingual translation tools, and interactive speech-driven experiences. The `on_speech()` function from Part 7 is your starting point - customize it and make something amazing!